# Codificación del modelo GPT

![Texto alternativo](./imgs/4.15.png)

**Resumen  de  la  arquitectura  del  modelo  GPT.**  Esta  figura  ilustra  el  flujo  de  datos  a  través  del  modelo  GPT.  Comenzando  desde  abajo,  el  texto  tokenizado  se  convierte  primero  en  incrustaciones  de  token,  que  luego  se  amplían  con  incrustaciones  posicionales.  Esta  información  combinada  forma  un  tensor  que  pasa  por  una  serie  de  bloques  de  transformadores  que  se  muestran  en  el  centro  (cada  uno  con  capas  de  red  neuronal  de  atención  multicabezal  y  de  avance  con  deserción  y normalización  de  capas),  que  se  apilan  uno  sobre  el  otro  y  se  repiten  12  veces.

En  el  caso  del  modelo  GPT2  de  124  millones  de  parámetros,  se  repite  12  veces,  lo  cual  especificamos  mediante  la  entrada  "n_layers"  del  diccionario  GPT_CONFIG_124M .  En  el  caso  del  modelo  GPT2  más  grande,  con  1542  millones  de  parámetros,  este  bloque  de  transformación  se  repite  36  veces.

La  salida  del  bloque  transformador  final  pasa  por  un  paso  de normalización  de  capa  final  antes  de  llegar  a  la  capa  de  salida  lineal.  Esta  capa  asigna  la  salida  del  transformador  a  un  espacio  de  alta  dimensión  (en  este  caso,  50.257  dimensiones,  correspondiente  al  tamaño  del  vocabulario  del  modelo)  para  predecir  el  siguiente  token  en  la  secuencia.


In [18]:
from transformeBlock import TransformerBlock
from layerNorm import LayerNorm
from gptConfig124M import GPT_CONFIG_124M
import torch
import torch.nn as nn

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])  #50257 * 768
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device)) #  La  configuración  del  dispositivo  nos  permitirá  entrenar  el  modelo  en  una  CPU  o  GPU,  dependiendo  de  qué  dispositivo  sea  la  entrada.
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

El  constructor  __init__  de  esta  clase  GPTModel  inicializa  las  capas  de  incrustación  de  tokens  y  de  posición  utilizando  las  configuraciones  pasadas  mediante  un  diccionario  de  Python,  cfg.  Estas  capas  de  incrustación  se  encargan  de  convertir  los  índices  de  los  tokens  de  entrada  en  vectores  densos  y  de  añadir  información  posicional,  como  se  explica  en  el  capítulo  2.

A  continuación,  el  método  __init__  crea  una  pila  secuencial  de  módulos  TransformerBlock  igual  al  número  de  capas  especificado  en  cfg.  Tras  los  bloques  de  transformador,  se  aplica  una  capa  LayerNorm ,  que  estandariza  las  salidas  de  los  bloques  de  transformador  para  estabilizar  el  proceso  de  aprendizaje.  Finalmente,  se  define  un  cabezal  de  salida  lineal  sin  sesgo,  que  proyecta  la  salida  del  transformador  en  el  espacio  de  vocabulario  del  tokenizador  para  generar  logits  para  cada  token  del  vocabulario.

El  método  de  avance  toma  un  lote  de  índices  de  tokens  de  entrada,  calcula  sus  incrustaciones,  aplica  las  incrustaciones  posicionales,  pasa  la  secuencia  a  través  de  los  bloques  de  transformación,  normaliza  la  salida  final  y,  a  continuación,  calcula  los  logits,  que  representan  las  probabilidades  no  normalizadas  del  siguiente  token.  Convertiremos  estos  logits  en  tokens  y  salidas  de  texto  en  la  siguiente  sección.

In [19]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [20]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.1381,  0.0077, -0.1963,  ..., -0.0222, -0.1060,  0.1717],
         [ 0.3865, -0.8408, -0.6564,  ..., -0.5163,  0.2369, -0.3357],
         [ 0.6989, -0.1829, -0.1631,  ...,  0.1472, -0.6504, -0.0056],
         [-0.4290,  0.1669, -0.1258,  ...,  1.1579,  0.5303, -0.5549]],

        [[ 0.1094, -0.2894, -0.1467,  ..., -0.0557,  0.2911, -0.2824],
         [ 0.0882, -0.3552, -0.3527,  ...,  1.2930,  0.0053,  0.1898],
         [ 0.6091,  0.4702, -0.4094,  ...,  0.7688,  0.3787, -0.1974],
         [-0.0612, -0.0737,  0.4751,  ...,  1.2463, -0.3834,  0.0609]]],
       grad_fn=<UnsafeViewBackward0>)


Como se puede ver,  el  tensor  de  salida  tiene  la  forma  [2,  4,  50257],  ya  que se pasan  dos  textos  de  entrada  con  4  tokens  cada  uno.  
La  última  dimensión,  50257,  corresponde  al  tamaño  del  vocabulario  del  tokenizador. 

## Exploración de la arquitectura del modelo

In [21]:
#Recopilación del número de parámetros con el método numel
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


Anteriormente el modelo GPt era 124 millones de parámetros, pero el código anterior da un resultado de 163 millones.

La razon es el concepto **ligamento de pesos**.  Esto  significa  que  esta  arquitectura  reutiliza  los  pesos  de  la  capa  de  incrustación  de  tokens  en  su  capa  de  salida.

In [22]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

#Ambos tensores tienen la misma forma

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


Las  capas  de  incrustación  y  salida  de  tokens  son  muy  grandes  debido  al  número  de  filas  para  el  valor  50,257  en  el  vocabulario  del  tokenizador.  Eliminemos  el  recuento  de  parámetros  de  la  capa  de  salida  del  recuento  total  del  modelo  GPT2  según  la  vinculación  de  pesos:

In [23]:
total_params_gpt2 =  total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")

Number of trainable parameters considering weight tying: 124,412,160


La  vinculación  de  pesos  reduce  el  consumo  de  memoria  y  la  complejidad  computacional  del  modelo.  Sin  embargo,  según ,  el  uso  de  capas  separadas  de  incrustación  de tokens  y  de  salida  mejora  el  entrenamiento  y  el  rendimiento  del  modelo;  por  lo  tanto,  utilizamos  capas  separadas  en  nuestra  implementación  de  GPTModel .  Esto  mismo  ocurre  con  los  LLM  modernos.

### La Analogía del Diccionario Bilingüe
Imagina que estás construyendo un traductor. Necesitas dos diccionarios:

Diccionario de Palabra a Concepto (Embedding Layer): Toma una palabra en español ("gato") y la convierte a un concepto universal, una idea abstracta que no depende del idioma (un vector numérico).

"gato" → [0.1, -0.9, 0.5, ...] (El "significado" de gato en números)

Diccionario de Concepto a Palabra (Output Layer): Toma ese concepto universal ([0.1, -0.9, 0.5, ...]) y busca cuál es la palabra más probable en español que le corresponde.

[0.1, -0.9, 0.5, ...] → "gato"

La "ligación de pesos" es darse cuenta de que estos dos diccionarios son en realidad el mismo libro, solo que leído en direcciones opuestas. Si sabes cómo ir de "gato" a su concepto, también sabes cómo ir del concepto a "gato".

En lugar de crear y entrenar dos diccionarios gigantes por separado, el modelo usa exactamente la misma matriz de pesos para ambas tareas.

In [24]:
#Requisitos de memoria de los 163 millones de parámetros
total_size_bytes = total_params * 4            #Asumiento float32, 4 bytes por parámetros                  
total_size_mb = total_size_bytes / (1024 * 1024)         #1 MB es equivalente a 1.048.576 bytes         
print(f"Total size of the model: {total_size_mb:.2f} MB")


Total size of the model: 621.83 MB


En  conclusión,  al  calcular  los  requisitos  de  memoria  para  los  163  millones  de  parámetros  en  nuestro  objeto  GPTModel  y  asumiendo  que  cada  parámetro  es  un  flotante  de  32  bits  que  ocupa  4  bytes,  encontramos  que  el  tamaño  total  del  modelo  asciende  a  621,83  MB,  lo  que  ilustra  la  capacidad  de  almacenamiento  relativamente  grande  requerida  para  acomodar  incluso  LLM  relativamente  pequeños.

[Generación de texto](./7_generacion_texto.ipynb)